In [ ]:
# Import libraries
import os
import zipfile

# API functions
import requests
import pandas as pd
import datetime

In [ ]:
# Define Functions

def get_start_date():
  """Calculates the date for 6 months ago from the current date."""
  today = datetime.date.today()
  return today - datetime.timedelta(days = 30 * 6)  # 30 days in a month

def get_end_date():
  """Returns today's date."""
  today = datetime.date.today()
  return today - datetime.timedelta(days = 1)  # 30 days in a month

In [ ]:
# Calculate Dates
start_date = get_start_date()
end_date = get_end_date()

print(start_date)
print(end_date)

In [ ]:
# Format Dates
formatted_start_date = start_date.strftime("%Y-%m-%dT00:00:00+00:00")
formatted_end_date = end_date.strftime("%Y-%m-%dT23:59:59+00:00")

print(formatted_start_date)
print(formatted_end_date)

In [ ]:
# Build API URL
base_url = "https://api.esios.ree.es/archives"
date_type = "datos"
url = f"{base_url}?start_date={formatted_start_date}&end_date={formatted_end_date}&date_type={date_type}"

print(url)

In [ ]:
# Authentication Setup
api_key = "c965de4dda51fe59aa6c522dcb519ed794e0082a3abe70d73ebd491c463da433"
headers = {
    "Accept": "application/json; application/vnd.esios-api-v1+json",
    "Content-Type": "application/json",
    "x-api-key": api_key
}

# Send API Request
response = requests.get(url, headers = headers)
data = response.json()
print(data)

In [ ]:
# Accessing the Download Link
download_link = data['archives'][0]['download']['url']
print(download_link)

In [ ]:
base_url = "https://api.esios.ree.es"
complete_download_link = f"{base_url}{download_link}"

# Define the data folder path
data_folder = "Data"

# Check if the folder exists, create it if not
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    
# Downloading the Data:
response = requests.get(complete_download_link)

if response.status_code == 200:
    # Download successful!
    data_content = response.content  # Get the downloaded data as bytes

    # Save the data to a file
    with open('Data/downloaded_data.zip', 'wb') as f:
        f.write(data_content)
    print("Data downloaded successfully!")
else:
    print(f"Error downloading data: {response.status_code}")

In [ ]:
try:
  # Open the zip file
  with zipfile.ZipFile('Data/downloaded_data.zip', 'r') as zip_ref:
    # Extract all files to the data folder
    zip_ref.extractall(data_folder)
    print("Data extracted successfully!")
except zipfile.BadZipFile as e:
  print(f"Error extracting data: {e}")
except Exception as e:  # Catch other potential exceptions
  print(f"Unexpected error: {e}")

In [ ]:
def extract_nested_zip_files(nested_zip_filenames, data_folder, target_filename_pattern):
    extracted_files = []

    for filename in nested_zip_filenames:
        if filename.startswith("C2_PrecioFinal") and filename.endswith(".zip"):
            # Extract nested zip temporarily within data_folder
            nested_zip_file = os.path.join(data_folder, filename)  # Use os.path.join for joining paths
            outer_zip.extract(filename, data_folder)  # Extract using the copied filename

            with zipfile.ZipFile(nested_zip_file, 'r') as inner_zip:
                for inner_info in inner_zip.infolist():
                    if inner_info.filename.startswith(target_filename_pattern):
                        extracted_files.append(inner_info.filename)
                        inner_zip.extract(inner_info.filename, data_folder)

            # os.remove(nested_zip_file)  # Remove the extracted nested zip

    if extracted_files:
        print("Extracted files:")
        for filename in extracted_files:
            print(f"- {filename}")
    else:
        print("No files matching the target pattern were found.")
        
nested_zip_filenames = []
with zipfile.ZipFile('Data/downloaded_data.zip', 'r') as outer_zip:
  for info in outer_zip.infolist():
    if info.filename.startswith("C2_PrecioFinal") and info.filename.endswith(".zip"):
      print(f"Found nested zip file: {info.filename}")  # Print for verification
      nested_zip_filenames.append(info.filename)  # Store filenames
  
  # Define target pattern
  target_filename_pattern = "PFMHORAS_TOD_*"
  # Extract files
  extract_nested_zip_files(nested_zip_filenames, data_folder, target_filename_pattern)